## Signal Amplification by Reversible Exchange (SABRE) - Pyridine: Three Spins

This example demonstrates a simple SABRE simulation of pyridine. Key details of the simulation are as follows:

- **Spin System**: The system consists of two hydride protons and one ligand proton.
- **Simplifications**: Chemical exchange and relaxation effects are not included in this simulation.
- **Simulation Conditions**: The simulation is performed near the energy level anticrossing field, leading to coherent oscillations between quantum states.
- **Performance**: The example runs in a few seconds on a laptop with an 11th-generation i5 processor.

This example provides an accessible introduction to SABRE simulations while focusing on the essential dynamics of the spin system.

### Imports

- **NumPy**: Used for creating and manipulating arrays.
- **Matplotlib**: Used for plotting data and visualizing results.
- **Spinguin**: Provides the core functionality for simulating spin systems, including Hamiltonian calculations, propagators, and measurements.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from spinguin import SpinSystem, hamiltonian, propagator, singlet, measure

### Simulation Parameters

The following simulation parameters are defined:

- **Magnetic Field**: Set to 5.3 mT.
- **Time Step**: Set to 1 ms.
- **Number of Time Steps**: Set to 1000, resulting in a total simulation time of 1 second.

In [2]:
# Define simulation parameters
B0 = 5.3e-3     # Magnetic field strength in Tesla (5.3 mT)
dt = 1e-3       # Time step for the simulation in seconds (1 ms)
N_steps = 1000  # Number of time steps (total simulation time = 1 second)

### Define the Spin System

To perform the simulations, we need to define the spin system.

- **Isotopes**: The types of nuclei in the spin system.
- **Chemical Shifts**: Specified in parts per million (ppm).
- **Scalar Couplings**: Specified in Hertz (Hz).

These parameters are defined as NumPy arrays. Additional parameters, such as molecular coordinates or shielding tensors, are not included since the simulation does not involve relaxation effects. Once the required parameters are defined, they are passed to the spin system constructor.

In [3]:
# Define the spin system parameters
isotopes = np.array(['1H', '1H', '1H'])  # Three hydrogen nuclei
chemical_shifts = np.array([-22.7, -22.7, 8.34])  # Chemical shifts for each nucleus
J_couplings = np.array([
    [ 0,     0,      0],    # Coupling matrix (Hz)
    [-6.53,  0,      0],    # Coupling between nuclei
    [ 0.00,  1.66,   0]
])

# Create the spin system using the defined parameters
spin_system = SpinSystem(isotopes, chemical_shifts, J_couplings)

## Calculate the Hamiltonian and the Time Propagator

The following steps are performed in this section:

- **Hamiltonian**: The Hamiltonian is calculated for the defined spin system. The magnetic field strength is passed as a parameter to the `hamiltonian` function.
- **Time Propagator**: Using the Hamiltonian and the defined time step, the time propagator is computed. This propagator will be used to evolve the spin system over time.

In [ ]:
# Calculate the Hamiltonian for the spin system
H = hamiltonian(spin_system, B0)

# Calculate the time propagator
P = propagator(dt, H)

### Assign the Initial State

The initial state for this simulation is defined as follows:

- **Hydride Spins**: The hydride spins are initialized in a singlet state.
- **Substrate Spin**: The substrate spin is assumed to be in thermal equilibrium, with negligible magnetization.

In [5]:
# Initialize the density matrix
rho = singlet(spin_system, 0, 1)

### Create an Empty Array for Storing Magnetizations

To store the magnetizations during the simulation, we need to create an empty array.

- The simulation involves 1000 time steps.
- After each time step, the magnetization of each spin will be calculated.
- The array will store the magnetizations for all spins at each time step.

In [6]:
# Create an empty array to store magnetizations
magnetizations = np.empty((N_steps, spin_system.size), dtype=complex)

### Perform the Time Evolution

- Loop over the defined number of time steps.
- Propagate the spin system forward in time during each step using the time propagator.
- Calculate the magnetization for each spin at each time step.
- Save the calculated magnetizations to the previously created array for later analysis.

In [7]:
# Perform the time evolution of the spin system
for step in range(N_steps):
    # Propagate the density matrix to the next time step
    rho = P @ rho
    
    # Measure the magnetization of each spin and store it in the magnetizations array
    for i in range(isotopes.size):
        magnetizations[step, i] = measure(spin_system, rho, 'I_z', i)

### Plot the Magnetizations and Visualize the Results

- Calculate the time axis for the simulation.
- Plot the magnetizations of each spin as a function of time.
- Observe that the substrate spin achieves near full magnetization after approximately 0.5 seconds.

In [ ]:
# Create a time axis for the simulation
t = np.linspace(dt, N_steps * dt, N_steps)

# Plot the magnetizations for each spin as a function of time
for i in range(isotopes.size):
    # Plot the real part of the magnetization for spin i
    plt.plot(t, np.real(magnetizations[:, i]), label=f"Spin {i+1}")

# Add a legend to identify each spin
plt.legend(loc="upper right")

# Add labels and title to the plot for clarity
plt.xlabel("Time (s)")  # Label for the x-axis (time in seconds)
plt.ylabel("Magnetization")  # Label for the y-axis (magnetization values)
plt.title("SABRE-Hyperpolarization of Pyridine")  # Title of the plot

# Adjust layout to prevent overlapping elements and display the plot
plt.tight_layout()
plt.show()

# Clear the figure to avoid overlapping plots in subsequent cells
plt.clf()